In [21]:
import json
from collections import defaultdict
import altair as alt
alt.data_transformers.disable_max_rows()
import numpy as np
alt.data_transformers.enable('data_server')
alt.renderers.enable('default')
import pandas as pd
from itertools import groupby
from collections import OrderedDict
import copy
from itertools import combinations
from collections import Counter
from sklearn.datasets import make_classification,load_iris
from imblearn.under_sampling import RandomUnderSampler 
import math
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score,average_precision_score
import pickle
import sklearn.pipeline
import xgboost as xgb
import xmltodict

# codes from https://github.com/gerritjandebruin/temporal-link-prediction/blob/main/src/get_features.py
#. for temporal link prediction
import collections
import itertools
import os

import networkx as nx
import scipy.stats
import typer

import logger
from progress_parallel import ProgressParallel, delayed

app = typer.Typer()

### calculate temporal features with cn, jc, aa
- codes revised from https://github.com/gerritjandebruin/temporal-link-prediction/blob/67b1eac9dd984a218be6603b254f0fe079c20c93/src/get_features.py
- feature vector length == weighting functions * past event aggregation * topological feature==3 * 8 * 3

In [22]:
save_path = "/Users/yidesdo21/Projects/outputs/13_link_prediction/"

In [23]:
feature_files = os.listdir(save_path)
scores = dict()   # {score_file_name:a_list_of_scores, ...}

for feature_file in feature_files:
    if feature_file != ".DS_Store":
        with open(save_path+feature_file,"rb") as fp:  # uncpickling
            f = pickle.load(fp)
            scores[feature_file] = f

In [24]:
# unpickling 
## X_res: for training data, sampled node pairs names, with 1:1 positive and negative labels
## y_res: for training data, labels for the node pairs
## X_res_tt: for testing data, sampled node pairs names, with 1:1 positive and negative labels
## y_res_tt: for testing data, labels for the node pairs
X_res,y_res,X_res_tt,y_res_tt = scores.get("x_res_gt_2019"),scores.get("y_res_gt_2019"),\
                                scores.get("x_res_tt_gt_2019"),scores.get("y_res_tt_gt_2019")

In [25]:
ver_idx = scores.get("vertices_global")

In [26]:
# load the metadata from the json
meta_path = "/Users/yidesdo21/Projects/outputs/12_time_slicing/metadata/"

with open(meta_path+"articles_with_anno.json") as f:
    metadata = json.load(f)

In [7]:
# with open(save_path+"cn_temp_gt_2019_sqrt_m3","rb") as fp:  # uncpickling
#     f = pickle.load(fp)

In [8]:
# f

In [27]:
# strategies
## codes revised from https://github.com/gerritjandebruin/temporal-link-prediction/blob/main/src/get_features.py
def _rescale(x: pd.Series, *, lower_bound: float = 0.2) -> pd.Series:
    """_rescale the provided array.
    Args:
      lower_bound: Instead of normalizing between 0 and 1, normalize between 
        lower_bound and 1.
    """
    lowest, highest = np.quantile(x, [0, 1])
    return lower_bound + (1-lower_bound)*(x-lowest)/(highest-lowest)


def _exp_time(x: pd.Series) -> pd.Series:
    """Apply y=3*exp(x) and normalize it between (0,1)."""
    return np.exp(3*x) / np.exp(3)


def lin(x: pd.Series, lower_bound=.2):
    return _rescale(_rescale(x.astype(int)), lower_bound=lower_bound)


def exp(x: pd.Series, lower_bound=.2):
    return _rescale(_exp_time(_rescale(x.astype(int))), lower_bound=lower_bound)


def sqrt(x: pd.Series, lower_bound=.2):
    return _rescale(np.sqrt(_rescale(x.astype(int))), lower_bound=lower_bound) #type: ignore


TIME_STRATEGIES = {'lin': lin, 'exp': exp, 'sqrt': sqrt}

AGGREGATION_STRATEGIES = {
    'q0': np.min,
    'q25': lambda array: np.quantile(array, .25),
    'q50': np.median,
    'q75': lambda array: np.quantile(array, .75),
    'q100': np.max,
    'm0': np.sum,
    'm1': np.mean,
    'm2': np.var,
#     'm3': scipy.stats.skew,      #### m3,m4 is very slow
#     'm4': scipy.stats.kurtosis
}

In [28]:
def get_instances(X_res,vertices):
    """create unconnected node pairs in the feature network and undersampled in the label network.,
            1:1 pos:neg ratio, indices for nodes,
       input: X_res -- 2D array of strings,
            vertices -- {entity_name:entity_index}
       output: instances -- a list of tuples"""
    instances = list()
    X_res_list = X_res.reshape(-1)
    vertices_inv = dict((v,k) for k,v in vertices.items())  # {entity_index:entity_name}
    
    for node_pair in X_res_list:
        idx1,idx2 = int(node_pair.split('::')[0]),int(node_pair.split('::')[1])
        entity1,entity2 = vertices_inv.get(idx1),vertices_inv.get(idx2)           
        instances.append((entity1,entity2))
        
    return instances
        

In [29]:
def create_edgelist_mature(metadata, t_max):
    """create a dataframe of entity1, entity2, and the time observed the links between two entities,
        each (edge0,edge1,year) with different year stamp has to be stored in the edgelist,
        input -- metadata, metadata for each article, 
              -- t_max, max year, inclusive,
        output -- a dataframe with node pairs and the time stamp
        """
    edge_year = list()   # all edges and year, (edge1,edge2,year), need duplications, for the weights

    for m in metadata:   # the edges are on the citation level
        year, db = int(m.get("year")), m.get("sourcedb")
        
        if year <= t_max:
            if db == "NLM":
                nio_uids = m.get("nio_uids")
                ptc_uids = m.get("ptc_uids")

                if nio_uids is not None and ptc_uids is not None:
                    uids = sorted(nio_uids+ptc_uids)
                    co_occur = list(combinations(uids, 2))
                    for pair in co_occur:
                        edge_year.append((pair[0],pair[1],year))
                
    edgelist_mature = pd.DataFrame(edge_year, columns=['source', 'target', 'datetime'])    
    
    return edgelist_mature

In [30]:
def cn(info, instances, edgelist_mature, X_res,vertices=ver_idx, save_path=save_path):
    """calculated cn values without temporal information, the last timestamp for each link is used 
        input -- info -- saving information, e.g. "cn_gt_2019" 
                instances: negative cases,
                 edgelist_mature: [node1,node2,datetime] dataframe,
                 X_res: a list, sampled node pairs names, with 1:1 positive and negative labels                
        output -- cn scores: a list of int, the length is number of unconnected node pairs (instances)"""
    
    G = nx.from_pandas_edgelist(edgelist_mature[['source', 'target']])
    scores = [len(list(nx.common_neighbors(G, u, v))) for u, v in instances]
        
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp)

    return scores

In [31]:
def pa(info, instances, edgelist_mature, X_res,vertices=ver_idx, save_path=save_path):
    G = nx.from_pandas_edgelist(edgelist_mature[['source', 'target']])
    scores = [p for _, _, p in nx.preferential_attachment(G, instances)]
    
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp)
        
    return scores

In [32]:
def jc(info, instances, edgelist_mature, X_res,vertices=ver_idx, save_path=save_path):
    """calculated jc values without temporal information, the last timestamp for each link is used 
        input -- ... 
                X_res: a list, sampled node pairs names, with 1:1 positive and negative labels                
        output -- jc scores: a list of int, the length is number of unconnected node pairs (instances)"""
    G = nx.from_pandas_edgelist(edgelist_mature[['source', 'target']])
    scores = [p for _, _, p in nx.jaccard_coefficient(G, instances)]
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp)    
    
    return scores    

In [33]:
def aa(info, instances, edgelist_mature, X_res,vertices=ver_idx, save_path=save_path):
    """calculated aa values without temporal information, the last timestamp for each link is used 
        input -- ... 
                 X_res: a list, sampled node pairs names, with 1:1 positive and negative labels                
        output -- aa scores: a list of int, the length is number of unconnected node pairs (instances)"""
    graph_mature = nx.from_pandas_edgelist(edgelist_mature)
    scores = [p for _, _, p in nx.adamic_adar_index(graph_mature, instances)]
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp)  
        
    return scores

In [34]:
def cn_time_aware(info, instances, edgelist_mature, X_res, time_strategy,
                  aggregation_strategy, vertices=ver_idx, save_path=save_path):
    """calculated cn values with weighted temporal information, 
        input -- ...
                 X_res: a list, sampled node pairs names, with 1:1 positive and negative labels,
                 vertices: {entity_name:entity_index},
                 time _strategy: strategy used to weight time as the edge of the multigraph
                 aggregation_strategy: strategy used to aggregate the edges for the same node pair in the multigraph,
        output -- cn scores: a list of int, the length is number of unconnected node pairs (instances)"""
        
    df = edgelist_mature[['source', 'target', 'datetime']].assign(
        datetime=lambda x: time_strategy(x['datetime'])
    )
    
    # create a multigraph, it allows multiple edges between two entities
    # G[u][z].values(), format: 
    #. multiple edges for one node pair are saved as a dictionary of dictionaries, {edge:{"datetime":...}, ...}
    #.  ValuesView(AtlasView({0: {'datetime': 1.0000000000000002}, 1: {'datetime': 0.2}}))
    G = nx.from_pandas_edgelist(df, create_using=nx.MultiGraph, edge_attr=True)
    scores = list()
    
    for u, v in instances:
        score = [
            ## aggregation_strategy([e['datetime'] for e in G[u][z].values()])
            # use the aggregation function on the multiple edges between the same pair of entities
            ## aggregation_strategy([e['datetime'] for e in G[u][z].values()]) +
            ## aggregation_strategy([e['datetime'] for e in G[v][z].values()])
            # sum the aggregated results from (u,z) and (v,z) for (u,v)
            aggregation_strategy([e['datetime'] for e in G[u][z].values()]) +
            aggregation_strategy([e['datetime'] for e in G[v][z].values()])
            # each common neighbour has to be summed for the score of (u,v)
            for z in nx.common_neighbors(G, u, v)
        ]
        scores.append(sum(score))   
        
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp) 
        
    return scores



In [35]:
def pa_time_aware(info, instances, edgelist_mature, X_res, time_strategy,
                  aggregation_strategy, vertices=ver_idx, save_path=save_path):
    df = edgelist_mature[['source', 'target', 'datetime']].assign(
        datetime=lambda x: time_strategy(x['datetime'])
    )
    G = nx.from_pandas_edgelist(df, create_using=nx.MultiGraph, edge_attr=True)

    scores = list()
    for u, v in instances:
        all_u = [aggregation_strategy([e['datetime'] for e in G[u][a].values()])
                 for a in G[u]]
        all_v = [aggregation_strategy([e['datetime'] for e in G[v][b].values()])
                 for b in G[v]]
        scores.append(sum(all_u) * sum(all_v))
        
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp)    
        
    return scores

In [36]:
def jc_time_aware(info, instances, edgelist_mature, X_res, time_strategy,
                  aggregation_strategy, vertices=ver_idx, save_path=save_path):
    """calculated jc values with weighted temporal information, 
        input -- ...
                 X_res: a list, sampled node pairs names, with 1:1 positive and negative labels,
                 vertices: {entity_name:entity_index},
                 time _strategy: strategy used to weight time as the edge of the multigraph
                 aggregation_strategy: strategy used to aggregate the edges for the same node pair in the multigraph,
        output -- jc scores: a list of int, the length is number of unconnected node pairs (instances)"""
    
   # logger.debug(f'Start converting edgelist.')
#     print("start")
    df = edgelist_mature[['source', 'target', 'datetime']].assign(
        datetime=lambda x: time_strategy(x['datetime'])
    )
    G = nx.from_pandas_edgelist(df, create_using=nx.MultiGraph, edge_attr=True)
    
#     print("build the graph")
    scores = list()

#     print("start opening cn file")
#     with open(save_path+info.replace("jc","cn"),"rb") as fp:  # uncpickling
#         f = pickle.load(fp)
#     cnt = 0
#     print(cnt)
    
    for u, v in instances:
        # logger.debug('Get CN')
        # already have cn scores
        cn = [
            aggregation_strategy([e['datetime'] for e in G[u][z].values()]) +
            aggregation_strategy([e['datetime'] for e in G[v][z].values()])
            for z in nx.common_neighbors(G, u, v)
        ]
        
        
        # logger.debug('Get all activity of nodes')
        all_u = [aggregation_strategy([e['datetime'] for e in G[u][a].values()])
                 for a in G[u]]
        all_v = [aggregation_strategy([e['datetime'] for e in G[v][b].values()])
                 for b in G[v]]
        all_activity = sum(all_u) + sum(all_v)
        # logger.debug('Get score')
        
#         instance_cn = f[cnt]
#         print(instance_cn)
#         cnt += 1
#         score = instance_cn / all_activity if all_activity != 0 else 0
        score = sum(cn) / all_activity if all_activity != 0 else 0
        scores.append(score)
        
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp) 
        
    return scores

In [37]:
def aa_time_aware(info, instances, edgelist_mature, X_res, 
                  time_strategy, aggregation_strategy, vertices=ver_idx, save_path=save_path):
    """calculated aa values with weighted temporal information, 
        input -- ...
                 X_res: a list, sampled node pairs names, with 1:1 positive and negative labels,
                 vertices: {entity_name:entity_index},
                 time _strategy: strategy used to weight time as the edge of the multigraph
                 aggregation_strategy: strategy used to aggregate the edges for the same node pair in the multigraph,
        output -- aa scores: a list of int, the length is number of unconnected node pairs (instances)"""    

    df = edgelist_mature[['source', 'target', 'datetime']].assign(
        datetime=lambda x: time_strategy(x['datetime'])
    )
    
    G = nx.from_pandas_edgelist(df, edge_attr=True, create_using=nx.MultiGraph)
    
    scores = list()
    
    for u, v in instances:
        score = [
            aggregation_strategy([e['datetime'] for e in G[u][z].values()]) *
            aggregation_strategy([e['datetime'] for e in G[v][z].values()]) /
            np.log(len(G[z]))
            for z in nx.common_neighbors(G, u, v)
        ]
        scores.append(sum(score))
        
    with open(save_path+info, "wb") as fp:   #Pickling
        pickle.dump(scores, fp)         
    
    return scores

In [38]:
## instances in the feature network of the training data
## edges in the feature network of the training network
instance_train = get_instances(X_res,vertices=ver_idx)
edgelist_train = create_edgelist_mature(metadata, t_max=2018)

In [39]:
## instances in the feature network of the testing data
## edges in the feature network of the testing network
instance_test = get_instances(X_res_tt,vertices=ver_idx)
edgelist_test = create_edgelist_mature(metadata, t_max=2019)

In [80]:
# cn_scores = cn("cn_gt_2019", instance_train, edgelist_train, X_res,vertices=ver_idx)
# jc_scores = jc("jc_gt_2019", instance_train, edgelist_train, X_res=X_res)
# aa_scores = aa("aa_gt_2019", instance_train, edgelist_train, X_res=X_res)

In [27]:
# pa_score = pa("pa_gt_2019", instance_train, edgelist_train, X_res=X_res)
# pa_tt_scores = pa("pa_tt_gt_2019", instance_test, edgelist_test, X_res=X_res_tt)

In [94]:
# cn_tt_scores = cn("cn_tt_gt_2019", instance_test, edgelist_test, X_res=X_res_tt)
# jc_tt_scores = jc("jc_tt_gt_2019", instance_test, edgelist_test, X_res=X_res_tt)
# aa_tt_scores = aa("aa_tt_gt_2019", instance_test, edgelist_test, X_res=X_res_tt)

In [18]:
# ## 3*8*3, for the training network
# for time_str, time_func in TIME_STRATEGIES.items():
#     for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
#         if not os.path.exists(save_path+"cn_temp_gt_2019_"+time_str+"_"+agg_str):
#             cn_temp_scores = cn_time_aware("cn_temp_gt_2019_"+time_str+"_"+agg_str, 
#                                            instance_train, edgelist_train, X_res=X_res,
#                                time_strategy=time_func,
#                                aggregation_strategy=agg_func)


In [40]:
for time_str, time_func in TIME_STRATEGIES.items():
    for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
        if not os.path.exists(save_path+"jc_temp_gt_2019_"+time_str+"_"+agg_str):
            print(agg_str, time_str)
            jc_temp_scores = jc_time_aware("jc_temp_gt_2019_"+time_str+"_"+agg_str, 
                                           instance_train, edgelist_train, X_res=X_res,
                               time_strategy=time_func,
                               aggregation_strategy=agg_func)

q25 sqrt
q50 sqrt
q75 sqrt
q100 sqrt
m0 sqrt


KeyboardInterrupt: 

In [38]:
# for time_str, time_func in TIME_STRATEGIES.items():
#     for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
#         if not os.path.exists(save_path+"pa_temp_gt_2019_"+time_str+"_"+agg_str):
#             print(agg_str, agg_func)
#             pa_temp_scores = pa_time_aware("pa_temp_gt_2019_"+time_str+"_"+agg_str, 
#                                            instance_train, edgelist_train, X_res=X_res,
#                                time_strategy=time_func,
#                                aggregation_strategy=agg_func)

q0 <function amin at 0x7fe1d0f34b80>
q25 <function <lambda> at 0x7fe05a98ab80>
q50 <function median at 0x7fe1d10511f0>
q75 <function <lambda> at 0x7fe05a98adc0>
q100 <function amax at 0x7fe1d0f349d0>
m0 <function sum at 0x7fe1d0f34160>
m1 <function mean at 0x7fe1d0f35790>
m2 <function var at 0x7fe1d0f35af0>


In [23]:
# ## 3*8*3, for the training network
# for time_str, time_func in TIME_STRATEGIES.items():
#     for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
#         if not os.path.exists(save_path+"aa_temp_gt_2019_"+time_str+"_"+agg_str):
#             aa_temp_scores = aa_time_aware("aa_temp_gt_2019_"+time_str+"_"+agg_str, 
#                                            instance_train, edgelist_train, X_res=X_res,
#                                time_strategy=time_func,
#                                aggregation_strategy=agg_func)

In [24]:
# ## 3*8*3, for the testing network
# for time_str, time_func in TIME_STRATEGIES.items():
#     for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
#         if not os.path.exists(save_path+"cn_temp_tt_gt_2019_"+time_str+"_"+agg_str):
#             cn_temp_scores_tt = cn_time_aware("cn_temp_tt_gt_2019_"+time_str+"_"+agg_str, 
#                                               instance_test, edgelist_test, X_res=X_res_tt,
#                                        time_strategy=time_func,
#                                        aggregation_strategy=agg_func)

In [ ]:
## 3*8*3, for the testing network
for time_str, time_func in TIME_STRATEGIES.items():
    for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
        print(agg_str, time_str)
        if not os.path.exists(save_path+"jc_temp_tt_gt_2019_"+time_str+"_"+agg_str):
            jc_temp_scores_tt = jc_time_aware("jc_temp_tt_gt_2019_"+time_str+"_"+agg_str, 
                                              instance_test, edgelist_test, X_res=X_res_tt,
                                       time_strategy=time_func,
                                       aggregation_strategy=agg_func)

In [25]:
# ## 3*8*3, for the testing network
# for time_str, time_func in TIME_STRATEGIES.items():
#     for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
#         if not os.path.exists(save_path+"aa_temp_tt_gt_2019_"+time_str+"_"+agg_str):
#             aa_temp_scores_tt = aa_time_aware("aa_temp_tt_gt_2019_"+time_str+"_"+agg_str, 
#                                               instance_test, edgelist_test, X_res=X_res_tt,
#                                        time_strategy=time_func,
#                                        aggregation_strategy=agg_func)

In [23]:
# for time_str, time_func in TIME_STRATEGIES.items():
#     for agg_str, agg_func in AGGREGATION_STRATEGIES.items():  
#         if not os.path.exists(save_path+"pa_temp_tt_gt_2019_"+time_str+"_"+agg_str):
#             print(agg_str, time_str)
#             pa_temp_scores = pa_time_aware("pa_temp_tt_gt_2019_"+time_str+"_"+agg_str, 
#                                            instance_test, edgelist_test, X_res=X_res_tt,
#                                time_strategy=time_func,
#                                aggregation_strategy=agg_func)

q0 exp
q25 exp
q50 exp
q75 exp
q100 exp
m0 exp
m1 exp
m2 exp
q0 sqrt
q25 sqrt
q50 sqrt
q75 sqrt
q100 sqrt
m0 sqrt
m1 sqrt
m2 sqrt
